In [ ]:
!pip install fsspec
!pip install gcsfs
!pip install gcloud
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.4 MB/s 
     |████████████████████████████████| 144 kB 64.9 MB/s 
     |████████████████████████████████| 271 kB 49.2 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 454 kB 4.2 MB/s 
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602937 sha256=124ffb0cca1f3de63a59f62f652cc7dbe462f824d169e7aad2595722e227c300
  Stored in directory: /root/.cache/pip/wheels/50/9d/19/dc1a639905431252dbf8a61388a83459c5cfef7add445bebe4
Successfully built gcloud
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/cola

In [ ]:
import pandas as pd
import fsspec
import gcsfs
import nltk
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, f1_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, HalvingGridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem.porter import *
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
# Paths to CSV files
path_to_gcs_1 = 'https://storage.googleapis.com/csv-etl-fyp/model/training/cnbc_headlines.csv'
path_to_gcs_2 = 'https://storage.googleapis.com/csv-etl-fyp/model/training/guardian_headlines.csv'
path_to_gcs_3 = 'https://storage.googleapis.com/csv-etl-fyp/model/training/reuters_headlines.csv'

In [ ]:
df_1 = pd.read_csv(path_to_gcs_1)
df_2 = pd.read_csv(path_to_gcs_2)
df_3 = pd.read_csv(path_to_gcs_3)

In [ ]:
# Concat 3 DataFrames to One
dataframes = [df_1, df_2, df_3]
df = pd.concat(dataframes)
len(df)

53650

In [ ]:
# Preprocessing 1 - Drop NA/Null Values

df = df.dropna()
df.head()

,Headlines,Time,Description
0,Jim Cramer: A better way to invest in the Covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,Cramer's lightning round: I would own Teradyne,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
3,"Cramer's week ahead: Big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,IQ Capital CEO Keith Bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."
5,Wall Street delivered the 'kind of pullback I'...,"7:36 PM ET Thu, 16 July 2020","""Look for the stocks of high-quality companies..."


In [ ]:
# Preprocessing 2 - Lowercase

df['Headlines'] = [x.lower() for x in df['Headlines']]

In [ ]:
# Preprocessing 3 - Stemming (PorterStemmer)

st = PorterStemmer()
df['Headlines'] = [st.stem(x) for x in df['Headlines']]
df

,Headlines,Time,Description
0,jim cramer: a better way to invest in the covi...,"7:51 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer recommended buying..."
1,cramer's lightning round: i would own teradyn,"7:33 PM ET Fri, 17 July 2020","""Mad Money"" host Jim Cramer rings the lightnin..."
3,"cramer's week ahead: big week for earnings, ev...","7:25 PM ET Fri, 17 July 2020","""We'll pay more for the earnings of the non-Co..."
4,iq capital ceo keith bliss says tech and healt...,"4:24 PM ET Fri, 17 July 2020","Keith Bliss, IQ Capital CEO, joins ""Closing Be..."
5,wall street delivered the 'kind of pullback i'...,"7:36 PM ET Thu, 16 July 2020","""Look for the stocks of high-quality companies..."
...,...,...,...
32765,malaysia says never hired british data firm at...,Mar 20 2018,The Malaysian government and the ruling party ...
32766,prosecutors search volkswagen headquarters in ...,Mar 20 2018,German prosecutors said on Tuesday they had se...
32767,mcdonald's sets greenhouse gas reduction target,Mar 20 2018,McDonald's Corp on Tuesday announced an approv...
32768,pratt & whitney to deliver spare a320neo engin...,Mar 20 2018,Pratt & Whitney will soon begin deliveries of ...


In [ ]:
# Preprocessing 4 - Stopwords & Punctuation removal

from nltk.corpus import stopwords
nltk.download('stopwords')
import string
stop = set(stopwords.words('english'))
df['Headlines'] = [t for t in df['Headlines'] if t not in stop and t not in string.punctuation]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# NLTK Sentiment Analyzer for Training data labels
sia = SentimentIntensityAnalyzer()

df['compound'] = [sia.polarity_scores(x)['compound'] for x in df['Headlines']]


In [ ]:
# Drop unused columns
df = df.drop(['Time', 'Description'], axis=1)

In [ ]:
# Sentiment Threshold (>0.05 = Positive, >-0.05 and <0.05 = Neutral, <0.05 = Negative)
df['sentiment'] = ['pos' if x >= 0.05 else 'neu' if x >-0.05 and x <0.05 else 'neg' for x in df['compound']]

In [ ]:
df.head()

,Headlines,compound,sentiment
0,jim cramer: a better way to invest in the covi...,0.4404,pos
1,cramer's lightning round: i would own teradyn,0.0000,neu
3,"cramer's week ahead: big week for earnings, ev...",0.0000,neu
4,iq capital ceo keith bliss says tech and healt...,0.5719,pos
5,wall street delivered the 'kind of pullback i'...,0.0000,neu


In [ ]:

# Fit CountVectorizer for XGBoost Input
cv = CountVectorizer(binary = True)
train = cv.fit_transform(df['Headlines'])

In [ ]:
# Train/Test Split
xsplit, xtest, ysplit, ytest = train_test_split(train, df['sentiment'], test_size=0.20, random_state=42)

In [ ]:
# Train/Validation Split
xtrain, xval, ytrain, yval = train_test_split(xsplit, ysplit, test_size=0.20, random_state=33)

In [ ]:
# Base XGB model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(xtrain, ytrain)

XGBClassifier(objective='multi:softprob')

In [ ]:
pred = xgb_model.predict(xval)

In [ ]:
pred_probability = xgb_model.predict_proba(xval)
pred = xgb_model.predict(xval)


In [ ]:
roc_auc_score(yval, pred_probability, multi_class="ovr")

0.8160843497886366

In [ ]:
# Function for model evaluation

def model_eval_wrapper(y_test, y_pred_test, xgb_clf, clf=False):\

    print("model Prediction : \n %s" %(y_pred_test[:5]))
    print("Expected Prediction: \n %s" %(y_test.head().values))
   
    if clf:
        accuracy = accuracy_score(y_test, y_pred_test)
        print("Accuracy : %.2f%%" % (accuracy * 100.0))
        F1 = f1_score(y_test,y_pred_test, average='macro')
        print("F1_score : %.2f%%" % (F1 * 100.0))
        precision = precision_score(y_test, y_pred_test, average='macro')
        print("Precision : %.2f%%" % (precision * 100.0))
        recall = recall_score(y_test, y_pred_test, average='macro')
        print("Recall : %.2f%%" % (recall * 100.0))
        print("Confusion Matrix : \n %s" %(confusion_matrix(y_test, y_pred_test)))
        
        return pd.DataFrame(classification_report(y_test,y_pred_test, output_dict=True))

In [ ]:
model_eval_wrapper(yval, pred, xgb_model, clf=True)

model Prediction : 
 ['neu' 'neu' 'neu' 'neg' 'pos']
Expected Prediction: 
 ['pos' 'neu' 'neg' 'pos' 'pos']
Accuracy : 66.29%
F1_score : 63.92%
Precision : 75.19%
Recall : 63.10%
Confusion Matrix : 
 [[ 765  828  121]
 [  39 2174   52]
 [  92  787  834]]


,neg,neu,pos,accuracy,macro avg,weighted avg
precision,0.853795,0.573766,0.828203,0.66286,0.751921,0.734662
recall,0.446324,0.959823,0.486865,0.66286,0.631004,0.662860
f1-score,0.586207,0.718203,0.613235,0.66286,0.639215,0.646866
support,1714.000000,2265.000000,1713.000000,0.66286,5692.000000,5692.000000


N_estimator Hyperparameter Tuning

In [ ]:
param_grid_nestimator = {
    'n_estimators': [400, 800, 1200, 2000]
}

In [ ]:
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_nestimator,
    scoring = 'f1_macro',
    n_jobs = -1,
    cv = 3,
    verbose=True
)
grid_search.fit(xsplit, ysplit)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


KeyboardInterrupt: ignored

In [ ]:
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

In [ ]:
plt.errorbar(param_grid_nestimator['n_estimators'], means, yerr=stds)
plt.title("XGBoost n_estimators / F1 score")
plt.xlabel('n_estimators')
plt.ylabel('F1 Score')

Max_Depth Hyperparameter Tuning

In [ ]:
param_grid_maxdepth = {
    'max_depth': [5, 10, 25, 35]
}

In [ ]:
grid_search_md = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_maxdepth,
    scoring = 'f1_macro',
    n_jobs = -1,
    cv = 3,
    verbose=True
)
grid_search_md.fit(xsplit, ysplit)

In [ ]:
means = grid_search_md.cv_results_['mean_test_score']
stds = grid_search_md.cv_results_['std_test_score']
params = grid_search_md.cv_results_['params']

In [ ]:
plt.errorbar(param_grid_maxdepth['max_depth'], means, yerr=stds)
plt.title("XGBoost max_depth / F1 score")
plt.xlabel('max_depth')
plt.ylabel('F1 Score')

Learning Rate

In [ ]:
param_grid_lr = {
  'learning_rate' : [0.05, 0.10, 0.20]
}

In [ ]:
grid_search_md = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_lr,
    scoring = 'f1_macro',
    n_jobs = -1,
    cv = 3,
    verbose=True
)
grid_search_md.fit(xsplit, ysplit)

In [ ]:
grid_search_md.best_params_

Final Model

In [ ]:
xgb_model = xgb.XGBClassifier(learning_rate=0.2, n_estimator=2000, max_depth=35)
xgb_model.fit(xtrain, ytrain)

XGBClassifier(learning_rate=0.2, max_depth=35, n_estimator=2000,
              objective='multi:softprob')

In [ ]:
pred = xgb_model.predict(xtest)
model_eval_wrapper(ytest, pred, xgb_model, clf=True)

model Prediction : 
 ['pos' 'neg' 'pos' 'neu' 'neu']
Expected Prediction: 
 ['pos' 'neg' 'pos' 'neu' 'neu']
Accuracy : 86.44%
F1_score : 85.98%
Precision : 87.26%
Recall : 85.34%
Confusion Matrix : 
 [[1647  315  148]
 [  61 2752   70]
 [ 128  243 1750]]


,neg,neu,pos,accuracy,macro avg,weighted avg
precision,0.897059,0.831420,0.889228,0.864352,0.872569,0.868123
recall,0.780569,0.954561,0.825083,0.864352,0.853404,0.864352
f1-score,0.834769,0.888745,0.855955,0.864352,0.859823,0.862960
support,2110.000000,2883.000000,2121.000000,0.864352,7114.000000,7114.000000


In [ ]:
# ROC
pred_probability = xgb_model.predict_proba(xtest)
roc_auc_score(ytest, pred_probability, multi_class="ovr")

0.9454165908894089

Pickling to GCS

In [ ]:
import pickle
model_file = open('xgb-base.pkl', 'wb')
pickle.dump(xgb_model, model_file)
model_file.close()

In [ ]:
from google.cloud import storage
from google.oauth2 import service_account
import os

# Load XGB Model to GCS

path = '/content/final-347314-04047fbe1c8e.json'
credentials = service_account.Credentials.from_service_account_file(path)
client = storage.Client(credentials=credentials, project='final-347314')
bucket = client.get_bucket('csv-etl-fyp')
blob = bucket.blob('model/ml/xgb-base.pkl')
blob.upload_from_filename('/content/xgb-base.pkl')

In [ ]:
blob.public_url

'https://storage.googleapis.com/csv-etl-fyp/model/ml/xgb-base.pkl'

In [ ]:
# Load CountVectorizer to GCS

cv_file = open('cv.pkl', 'wb')
pickle.dump(cv, cv_file)
cv_file.close()

In [ ]:

bucket = client.get_bucket('csv-etl-fyp')
blob = bucket.blob('model/cv/cv.pkl')
blob.upload_from_filename('/content/cv.pkl')

In [ ]:
# Retrieval Test

blob.download_to_filename('/content/loaded-xgb.pkl')
loaded_file = open('/content/loaded-xgb.pkl', 'rb')
mdl_ = pickle.load(loaded_file)
pred2 = mdl_.predict(xtest)
accuracy_score(ytest, pred2)

0.8643519820073096